# Quotations in Salesforce
Dado que ya en Ichigo se hará el quoting engine, ya no es necesario que la gente tenga más incentivos a cargar los datos pues será obligatorio. Lo que ahora necesitamos es una forma de integrar esto en Salesforce, por lo que voy a mandar los items que han quoteado los MPs a Salesforce a un nuevo objeto que voy a crear.

In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timezone
from my_apis.mb_connection import MetabaseConnection
from my_apis.sheets_functions import SheetsFunctions
from my_apis.sf_connection import SalesforceConnection, SalesforceFunctions

In [2]:
sf_credentials = '~/.keys/sf_credentials.json'
mb_credentials = '~/.keys/mb_credentials.json'

sfc = SalesforceConnection(os.path.expanduser(sf_credentials))
mbc = MetabaseConnection(os.path.expanduser(mb_credentials))
sff = SalesforceFunctions(sfc)

## Automation

In [3]:
# Sacamos los items ya existentes en salesforce
query = '''
select Name
from quoted_items__c
'''
sf_data = sfc.extract_data(query).rename({'Name':'name'}, axis=1).astype(int)

In [4]:
# Sacamos los de metabase
path = 'queries/quoted_items.sql'
with open(path, 'r') as f:
    query = f.read()
items = mbc.query_more_data(query, 'name')

In [5]:
# Procesamos y limpiamos los datos
upload_data = (
    items
    .merge(sf_data, on='name', how='outer', indicator=True)
    .query('_merge == "left_only"')
    .drop(['_merge'], axis=1)
    .replace({np.NaN:None})
    .replace({'target_price__c':0.0}, None)
    .assign(item_name__c=lambda x: x.item_name__c.apply(lambda df: df[:255]))
)

In [6]:
# Cargamos los cambios
errors = sff.add_related_records(
    df=upload_data,
    add_type='quoted_items__c',
    related_index_name='mp_account__c',
    print_every=10
)

Malformed request https://primaoperaciones.my.salesforce.com/services/data/v57.0/sobjects/quoted_items__c/. Response content: [{'message': 'invalid cross reference id', 'errorCode': 'INVALID_CROSS_REFERENCE_KEY', 'fields': []}]


## Preparation

#### Un quote en salesforce tendrá la siguiente información,
1. item_quote_id
2. rfq_id
3. rfq_name
4. item_id (nombre del id)
5. customer account (sf id del customer) (se cancela, solo pueden tener una account relacionada, no varias)
6. pod
7. main process
8. quoted_currency
9. quoted_price
10. target_currency (if target_price != 0)
11. target_price

In [ ]:
path = 'queries/quoted_items.sql'
with open(path, 'r') as f:
    query = f.read()
items = mbc.query_more_data(query, 'name')

In [ ]:
# Un quote en salesforce tendrá la siguiente información:
# Vamos a dar de alta los items en salesforce

upload_data = (
    items
    .replace({np.NaN:None})
    .replace({'target_price__c':0.0}, None)
)

In [ ]:
errors = sff.add_related_records(
    df=upload_data,
    add_type='quoted_items__c',
    related_index_name='mp_account__c',
    print_every=10
)